# Libraries

In [1]:
import gpflow
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import tensorflow as tf
%matplotlib inline

import csv
import time
import copy
import json
import pandas as pd
import scipy as sp
import scipy.stats
import random
import copy

# R magic
#import rpy2


# the following lines will allow us to convert between Pandas DataFrames and R DataFrames
#import rpy2.robjects as ro
#from rpy2.robjects import pandas2ri
#pandas2ri.activate()
#from rpy2.robjects.conversion import ri2py

# this loads the R magic extension
#%load_ext rpy2.ipython

C:\Users\panch\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Functions
Docs:
- [GP Regression](http://gpflow.readthedocs.io/en/latest/notebooks/regression.html)
- [Optimize](https://gpflow.readthedocs.io/en/latest/notebooks/models.html?highlight=optimize#Optimization)

### Aux functions:

In [2]:
# Transformation of array to matrix
def array_to_matrix(x):
    X = []
    for i in range(len(x)):
        X.append([float(x[i])])
    X = np.array(X)
    return X

In [3]:
def get_new_kernel(kernel_string):
    # Initial new non-optimized kernels
    l = gpflow.kernels.Linear(1)
    p = gpflow.kernels.Periodic(1)
    r = gpflow.kernels.RBF(1)
    
    if   kernel_string == 'l': return l
    elif kernel_string == 'p': return p
    elif kernel_string == 'r': return r

    elif kernel_string == 'l+r': return  l+r
    elif kernel_string == 'l+p': return  l+p
    elif kernel_string == 'p+r': return  p+r

    elif kernel_string == 'l*r': return  l*r
    elif kernel_string == 'l*p': return  l*p
    elif kernel_string == 'p*r': return  p*r

    elif kernel_string == 'l+p+r': return l+p+r
    elif kernel_string == 'l+p*r': return l+p*r
    elif kernel_string == 'l*r+p': return l*r+p
    elif kernel_string == 'l*p+r': return l*p+r
    elif kernel_string == 'l*p*r': return l*p*r
    
    else: return 'error'

In [4]:
def lml(model):
    """Log marginal likelihood of a GP"""
    
    try:
        return model.compute_log_likelihood()
    except:
        print('Exception caught in lml')
        return -999999999

In [5]:
def compute_lmls(models):
    models_data = {}
    for key in models.keys():
        e = {
            'lml': lml(models[key]['model']),
            'white_added': models[key]['white_added'],
            'second_exception': models[key]['second_exception']
            }
        models_data[key] = e
        
    return models_data

In [6]:
def compute_new_lmls(models, newX, newY0):
    
    newY = standard_y(newY0)
    
    models_data = {}
    for key in models.keys():
        
        model = models[key]['model']
        
        X = model.X.value
        
        # Using solution of https://github.com/GPflow/GPflow/issues/835 :
        fd = {model.X.parameter_tensor:newX, 
              model.Y.parameter_tensor:newY}
        
        e = {
            # Compute the likelihood given a new Y dataset
            'lml': model.compute_log_likelihood(feed_dict=fd)
            }
        models_data[key] = e
        
    return models_data

### Create a new model and optimize it:

In [7]:
def compute(X, Y, kernel_string):
    # Get kernel
    kernel = get_new_kernel(kernel_string)
    
    # Build model
    m = gpflow.models.GPR(X, Y, kern = kernel)
    
    # Auxiliary variables
    white_added = False
    second_exception = False
    
    # Try to optimize
    try:
        m.compile()
        opt = gpflow.train.ScipyOptimizer()
        opt.minimize(m)
        
    # If it fails, do something depending on whether it's the first time or not
    except:
        # If a White Kernel has not been added (meaning that this is the first exception occurring)
        if not white_added: #(this IF might be unnecessary)
            # Add white kernel
            white_added = True

            w = gpflow.kernels.White(1, variance = 0.05)
            #w.variance.fixed = True
            w.variance.trainable = False
            m = gpflow.models.GPR(X, Y, kern = kernel + w)
            
        # Try to optimize again
        try:
            print('Adding White Kernel to', kernel_string)
            m.compile()
            opt = gpflow.train.ScipyOptimizer()
            opt.minimize(m)
        
        # If it fails again
        except:
            second_exception = True
            print('Exception caught computing', kernel_string)
    
    return {
            'model': m ,
            'white_added': white_added,
            'second_exception': second_exception
           }

### Create models with all combinations

In [8]:
def standard_y(Y0):
    return Y0 - np.mean(Y0)

In [9]:
def compute_gps(X, Y0):
    # Subtract the mean
    Y = standard_y(Y0)
    
    gps = {}

    gps['l'] = compute(X, Y, 'l')
    gps['p'] = compute(X, Y, 'p')
    gps['r'] = compute(X, Y, 'r')

    gps['l+p'] = compute(X, Y, 'l+p')
    gps['l*p'] = compute(X, Y, 'l*p')
    
    gps['l+r'] = compute(X, Y, 'l+r')
    gps['l*r'] = compute(X, Y, 'l*r')
    
    gps['p+r'] = compute(X, Y, 'p+r')
    gps['p*r'] = compute(X, Y, 'p*r')

    gps['l+p+r'] = compute(X, Y, 'l+p+r')
    gps['l+p*r'] = compute(X, Y, 'l+p*r')
    gps['l*p+r'] = compute(X, Y, 'l*p+r')
    gps['l*r+p'] = compute(X, Y, 'l*r+p')
    gps['l*p*r'] = compute(X, Y, 'l*p*r')
    
    return gps

### Create models for a dataset:

In [10]:
def compute_gps_for_dataset(dataset, Xpredictions = None):
    t0 = time.time()
    ids = np.unique(dataset['id'])
    
    lmls = {}
    new_lmls = {}
    
    for i in ids:
        print(i)
        # Filter the relevant data
        filtered_data = dataset[dataset['id'] == i]
        
        # Get X and Y
        X = array_to_matrix(filtered_data['x'].values)
        Y = array_to_matrix(filtered_data['y'].values)
        
        # Compute GPs
        gps = compute_gps(X, Y)
        
        print('Compute OK')
        
        # Calculate the log marginal likelihoods
        likelihoods = compute_lmls(gps)
        print('LMLs OK')
        
        # Calculate NEW log marginal likelihoods, given a NEW dataset for Y
        new_likelihoods = compute_new_lmls(gps)
        
        # Save
        i = str(i)
        lmls[i] = likelihoods
        new_lmls[i] = new_likelihoods
        

    print('Minutes:', str(round((time.time() - t0) / 60)))
        
    return {
            'lmls': lmls,
            'new_lmls': new_lmls
           }

In [11]:
def compute_gps_for_dataset_with_original_list(dataset, original_plots):
    t0 = time.time()
    ids = np.unique(dataset['id'])
    
    lmls = {}
    new_lmls = {}
    
    for i in ids:
        print(i)
        # Filter the relevant data
        filtered_data = dataset[dataset['id'] == i]
        
        # Get X and Y of the participant's data
        X_new = array_to_matrix(filtered_data['x'].values)
        Y_new = array_to_matrix(filtered_data['y'].values)
        
        # Original plots, given the graph that the participant saw
        plot_id = filtered_data['shown_description_plot_id'].iloc[0] #First element bc all of them are the same one.
        orig_plot_data = original_plots[original_plots['plot_id'] == plot_id]
        
        X_original = array_to_matrix(orig_plot_data['x'].values)
        Y_original = array_to_matrix(orig_plot_data['y'].values)
        
        # Compute GPs for the original data
        gps = compute_gps(X_original, Y_original)
        
        print('Compute OK')
        
        # Calculate the log marginal likelihoods
        likelihoods = compute_lmls(gps)
        print('LMLs OK')
        
        # Calculate NEW log marginal likelihoods, given a NEW dataset for Y
        new_likelihoods = compute_new_lmls(gps, X_new, Y_new)
        
        # Calculate original and new LMLs
        #likelihoods = compute_lmls_and_new_lmls(gps, X_new, Y_new)
        
        # Save
        i = str(i)
        lmls[i] = likelihoods
        #new_lmls[i] = new_likelihoods
        

    print('Minutes:', str(round((time.time() - t0) / 60)))
        
    return {
            'lmls': lmls,
            'new_lmls': new_lmls
           }

## Participant's data (Experiment 2)

In [12]:
# Import dataset from file
participants_data_exp2 = pd.read_csv("../experiment_2_francisco/analysis/data/2018-april-30-export_responses.csv",
                        index_col = 0)

# Create ID column = "pid_plotid"
participants_data_exp2['id'] = participants_data_exp2['pid'].map(str) + '_' + participants_data_exp2['shown_description_plot_id'].map(str)

In [13]:
dataset = participants_data_exp2[participants_data_exp2['pid'] < 2]

# Participant's data (shown + drawn)
Experiment 2

Steps:

1- Compute a GP for the original plot.

2- Calculate the LML on that GP for the respective participant's data.

In [14]:
# 1- Original plots list
original_plots = pd.read_csv("../analysis/data/list_long_format.csv")

# 2- Participant's data
participants_data_exp2 = pd.read_csv("../experiment_2_francisco/analysis/data/2018-april-30-export_responses.csv",
                        index_col = 0)

# Create ID column = "pid_plotid"
participants_data_exp2['id'] = participants_data_exp2['pid'].map(str) + '_' + participants_data_exp2['shown_description_plot_id'].map(str)

dataset = participants_data_exp2

##### DEBUG ONLY: ##### 
dataset = participants_data_exp2[participants_data_exp2['pid'] < 2]
#######################

results = compute_gps_for_dataset_with_original_list(dataset, original_plots)

0_11


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 225.186573
  Number of iterations: 18
  Number of functions evaluations: 19


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 225.186573
  Number of iterations: 18
  Number of functions evaluations: 19


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 226.022816
  Number of iterations: 28
  Number of functions evaluations: 31


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 226.022816
  Number of iterations: 28
  Number of functions evaluations: 31


...

INFO:tensorflow:Optimization terminated with:
  Message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
  Objective function value: -216.109238
  Number of iterations: 31
  Number of functions evaluations: 119


INFO:tensorflow:Optimization terminated with:
  Message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
  Objective function value: -216.109238
  Number of iterations: 31
  Number of functions evaluations: 119


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 42.986668
  Number of iterations: 11
  Number of functions evaluations: 14


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 42.986668
  Number of iterations: 11
  Number of functions evaluations: 14


Compute OK
LMLs OK
Minutes: 89


In [15]:
results

{'lmls': {'0_11': {'l': {'lml': -225.1865729131238,
    'white_added': False,
    'second_exception': False},
   'p': {'lml': -226.02281636835738,
    'white_added': False,
    'second_exception': False},
   'r': {'lml': -73.25571680720522,
    'white_added': False,
    'second_exception': False},
   'l+p': {'lml': -221.81658746622588,
    'white_added': False,
    'second_exception': False},
   'l*p': {'lml': -225.18683976687666,
    'white_added': False,
    'second_exception': False},
   'l+r': {'lml': -73.2559388704533,
    'white_added': False,
    'second_exception': False},
   'l*r': {'lml': -159.12238278165447,
    'white_added': False,
    'second_exception': False},
   'p+r': {'lml': -70.32309083607781,
    'white_added': False,
    'second_exception': False},
   'p*r': {'lml': -73.25571696971961,
    'white_added': False,
    'second_exception': False},
   'l+p+r': {'lml': -70.34532162035461,
    'white_added': False,
    'second_exception': False},
   'l+p*r': {'lml': -73.2

In [33]:
results = compute_gps_for_dataset_with_original_list(dataset, original_plots)

NameError: name 'original_plots' is not defined

In [13]:
participants_data_exp2.head()

,pid,external_pid,age_range,full_date,gender,unix_epoch,page_index,shown_description_pid,shown_description_page_index,shown_description_plot_id,shown_description_description,x,y,id
0,0,-L9r6NFeTkSHQbSnEnaO,36-45,Wed Apr 11 2018 19:49:25 GMT-0500 (Eastern Sta...,female,1523494165264,2,10,3,11,It is a bunch of dots that goes up then down a...,0,5.0,0_11
1,0,-L9r6NFeTkSHQbSnEnaO,36-45,Wed Apr 11 2018 19:49:25 GMT-0500 (Eastern Sta...,female,1523494165264,2,10,3,11,It is a bunch of dots that goes up then down a...,25,10.0,0_11
2,0,-L9r6NFeTkSHQbSnEnaO,36-45,Wed Apr 11 2018 19:49:25 GMT-0500 (Eastern Sta...,female,1523494165264,2,10,3,11,It is a bunch of dots that goes up then down a...,50,7.5,0_11
3,0,-L9r6NFeTkSHQbSnEnaO,36-45,Wed Apr 11 2018 19:49:25 GMT-0500 (Eastern Sta...,female,1523494165264,2,10,3,11,It is a bunch of dots that goes up then down a...,75,10.0,0_11
4,0,-L9r6NFeTkSHQbSnEnaO,36-45,Wed Apr 11 2018 19:49:25 GMT-0500 (Eastern Sta...,female,1523494165264,2,10,3,11,It is a bunch of dots that goes up then down a...,100,2.6,0_11


### Example:

In [99]:
X = array_to_matrix([1,2,3])
Y = array_to_matrix([10,20,30])

#m = compute(X, Y, 'l+r')

models = compute_gps(X, Y)

#model = compute(X, Y, 'l')

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 10.556379
  Number of iterations: 22
  Number of functions evaluations: 23


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 10.556379
  Number of iterations: 22
  Number of functions evaluations: 23


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 9.863578
  Number of iterations: 125
  Number of functions evaluations: 158


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 9.863578
  Number of iterations: 125
  Number of functions evaluations: 158


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 10.556373
  Number of iterations: 17
  Number of functions evaluations: 18


...

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 10.552412
  Number of iterations: 21
  Number of functions evaluations: 22


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 10.556373
  Number of iterations: 24
  Number of functions evaluations: 31


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 10.556373
  Number of iterations: 24
  Number of functions evaluations: 31


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 10.552390
  Number of iterations: 31
  Number of functions evaluations: 36


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 10.552390
  Number of iterations: 31
  Number of functions evaluations: 36


In [102]:
new_lmls = compute_new_lmls(models, array_to_matrix([10,20,10]))

TypeError: Cannot interpret feed_dict key as Tensor: The name 'model.X.parameter_tensor' looks like an (invalid) Operation name, not a Tensor. Tensor names must be of the form "<op_name>:<output_index>".

In [77]:
dir(model['model'])

['X',
 'Y',
 '_Parentable__global_index',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_autoflow_compute_log_likelihood',
 '_build',
 '_build_likelihood',
 '_build_objective',
 '_build_predict',
 '_build_prior',
 '_clear',
 '_define_name',
 '_gen_index',
 '_get_node',
 '_index',
 '_is_param_like',
 '_likelihood_tensor',
 '_name',
 '_objective',
 '_parent',
 '_prior_tensor',
 '_read_index',
 '_replace_node',
 '_set_node',
 '_tensor_mode_parameter',
 '_update_node',
 'anchor',
 'as_pandas_table',
 'assign',
 'build',
 'build_objective',
 'childname',
 'chi

In [82]:
dir(model['model'].X)

['ParameterAttribute',
 '_Parentable__global_index',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_apply_transform',
 '_build',
 '_build_constrained',
 '_build_parameter',
 '_build_prior',
 '_check_tensor_trainable',
 '_clear',
 '_dataholder_tensor',
 '_define_name',
 '_externally_defined',
 '_fixed_shape',
 '_gen_index',
 '_index',
 '_init_parameter_attributes',
 '_init_parameter_defaults',
 '_init_parameter_value',
 '_initial_value_tensor',
 '_is_initialized_tensor',
 '_name',
 '_parameter_name',
 '_parent',
 '_read_index',
 '_read_parameter_tensor',
 

In [67]:
def predict(gps, X):
    predictions = {}
    
    # For every GP, build predictions
    for key in gps.keys():
        
        try:
            mean, var = gps[key]['model'].predict_y(X)
        except:
            print('Exception caught in predict')
            mean, var = np.array([0]), np.array([0])
        
        predictions[key] = {'mean': mean.tolist(), 
                            'var': var.tolist()}
        
    return predictions

In [69]:
## From Stackoverflow. https://stackoverflow.com/questions/31174295/getattr-and-setattr-on-nested-objects ##
import functools

def rsetattr(obj, attr, val):
    pre, _, post = attr.rpartition('.')
    return setattr(rgetattr(obj, pre) if pre else obj, post, val)

sentinel = object()
def rgetattr(obj, attr, default=sentinel):
    if default is sentinel:
        _getattr = getattr
    else:
        def _getattr(obj, name):
            return getattr(obj, name, default)
    return functools.reduce(_getattr, [obj]+attr.split('.'))
## End copy-paste ##

In [70]:
#s_kernel: string kernel
def get_kernel_attributes(s_kernel, results):
    x0 = s_kernel.find('kern')
    x1 = s_kernel.find('\x1b[1m') # end of the kernel name
    x2 = s_kernel.find('\x1b[0m') # end of variable name
    
    new_result = s_kernel[x0 : x1] + s_kernel[x1+4 : x2]
    
    results.append(new_result)
    
    new_s_kernel = s_kernel[(x2+1) : ]
    
    if new_s_kernel.find('kern') == -1:
        return results
    else:
        return get_kernel_attributes(new_s_kernel, results)

In [71]:
# GP Model
# String of the variable to change
def set_variable(kernel, s_variable, new_value, set_prior=False):
    #Remove the first part
    s_variable = s_variable[s_variable.find('.')+1 : ]
    
    if set_prior:
        s_variable = s_variable + '.prior'
    
    rsetattr(kernel, s_variable, new_value)

In [72]:
#Returns the value of a specific variable within a kernel
def get_value(kernel, s_variable):
    #Remove the first part
    s_variable = s_variable[s_variable.find('.')+1 : ]
    
    # Return the value
    return rgetattr(kernel, s_variable).value[0]

In [73]:
def get_full_bayesian_new_kernel(kernel_name, prior_gps):
    
    # The structure of this object is :
    # {'model': <GPflow.gpr.GPR object at 0x000001B2C940CFD0>, 'white_added': False, 'second_exception': False}
    gp_result = copy.deepcopy(prior_gps[kernel_name]) 
    
    kernel = gp_result['model'].kern
    
    # Get all the variables of the kernel
    all_variables = get_kernel_attributes(str(kernel), [])

    # Move the values to the Priors and reset the values
    for variable in all_variables:
        # Get the value
        value = get_value(kernel, variable)

        # Set the value in the prior
        prior = GPflow.priors.Gaussian(value, 1.)
        set_variable(kernel, variable, prior, set_prior = True)

        # Set the value to 1
        set_variable(kernel, variable, 1.)
    
    return kernel

In [74]:
def normalize(Y):
    std = np.std(Y)
    mu = np.mean(Y)
    
    return ((Y - mu)/std)
    

In [77]:
def gps_to_string(gps):
    strings = {}
    for key in gps.keys():
        strings[key] = str(gps[key])
        
    return strings

In [78]:
def dict_max(d):
    maxval = max(d.values())
    keys = [k for k,v in d.items() if v==maxval]
    return keys, maxval

In [79]:
def save_results(results, filename, new_format=True):
    
    if not new_format:
        with open('output/' + filename + '.json', 'w') as fp:
            json.dump(results, fp)
            
    else:
        #lmls: pid, composition, lml, white_added, second_exception
        with open('output/' + filename + '_lmls.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            
            #title
            writer.writerow(['pid', 'composition', 'lml', 'white_added', 'second_exception'])
            
            for pid in results['lmls']:
                for composition in results['lmls'][pid]:
                    writer.writerow([
                                     pid, 
                                     composition, 
                                     results['lmls'][pid][composition]['lml'],
                                     results['lmls'][pid][composition]['white_added'],
                                     results['lmls'][pid][composition]['second_exception']
                                    ])
        
        #Xpredictions #predictions
        #with open('output/' + filename + '_predictions.csv', 'w', newline='') as csvfile:
        #    writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            
            #title
        #    writer.writerow(['pid', 'composition', 'Xpredictions', 'predictions_mean', 'predictions_var'])
            
        #    for pid in results['predictions']:
        #        for composition in results['predictions'][pid]:
        #            for index, element in enumerate(results['predictions'][pid][composition]['mean']):
        #                writer.writerow([
        #                                 pid, 
        #                                 composition, 
        #                                 results['Xpredictions'][index][0],
        #                                 #results['predictions'][pid][composition]['mean'][index][0],
        #                                 #results['predictions'][pid][composition]['var'][index][0]
        #                                 results['predictions'][pid][composition]['mean'][index],
        #                                 results['predictions'][pid][composition]['var'][index]
        #                                ])
        

Compute Gaussian Process Models for a dataset

In [52]:
def compute_gps_for_dataset(dataset, Xpredictions = None, full_bayesian = False, dataset_posterior = []):
    t0 = time.time()
    
    ids = np.unique(dataset['id'])
    
    #gpss_objects = {}
    lmls = {}
    
    for i in ids:
        print(i)
        # Filter the relevant data
        filtered_data = dataset[dataset['id'] == i]
        
        # Get X and Y
        X = array_to_matrix(filtered_data['x'].values)
        Y = array_to_matrix(filtered_data['y'].values)
        
        # Compute GPs
        gps = compute_gps(X, Y)
        
        print('Compute OK')
        
        # Calculate the log marginal likelihoods
        likelihoods = compute_lmls(gps)
        print('LMLs OK')
        
        # Save
        i = str(i)
        #gpss_objects[i] = gps
        lmls[i] = likelihoods
        

    print('Minutes:', str(round((time.time() - t0) / 60)))
        
    return {
            #'gpss_objects': gpss_objects, #Actual objects
            'lmls': lmls
           }

# Data

### List of plots (experimental dataset)

In [81]:
Xpredictions = np.linspace(31, 1426, 280)[:,None]

In [82]:
# Import dataset from file
wide_data_list = pd.read_csv("../experiment_1_francisco/analysis/data/list.csv",
                        index_col = 'id')

# The data is in a matrix. Now transforming it into long-format
data_list = wide_data_list.stack().reset_index()

# Change column names
data_list = data_list.rename(columns={'id': 'id', 'level_1': 'x', 0: 'y'})

### GP DATA

In [62]:
results_list = compute_gps_for_dataset(data_list)
save_results(results = results_list, filename = 'plots_list')

0
Compute OK
LMLs OK
1
Compute OK
LMLs OK
2
Compute OK
LMLs OK
3
Compute OK
LMLs OK
4
Compute OK
LMLs OK
5
Compute OK
LMLs OK
6
Compute OK
LMLs OK
7
Compute OK
LMLs OK
8
Adding White Kernel to l*p+r
Compute OK
LMLs OK
9
Compute OK
LMLs OK
10
Compute OK
LMLs OK
11
Compute OK
LMLs OK
12
Compute OK
LMLs OK
13
Compute OK
LMLs OK
14
Compute OK
LMLs OK
15
Compute OK
LMLs OK
16
Compute OK
LMLs OK
17
Compute OK
LMLs OK
18
Compute OK
LMLs OK
19
Compute OK
LMLs OK
20
Compute OK
LMLs OK
21
Compute OK
LMLs OK
22
Compute OK
LMLs OK
23
Compute OK
LMLs OK
24
Compute OK
LMLs OK
25
Compute OK
LMLs OK
26
Compute OK
LMLs OK
27
Compute OK
LMLs OK
28
Compute OK
LMLs OK
29
Compute OK
LMLs OK
30
Compute OK
LMLs OK
31
Adding White Kernel to p+r
Compute OK
LMLs OK
32
Compute OK
LMLs OK
33
Compute OK
LMLs OK
34
Compute OK
LMLs OK
35
Compute OK
LMLs OK
36
Compute OK
LMLs OK
37
Compute OK
LMLs OK
38
Compute OK
LMLs OK
39
Compute OK
LMLs OK
Minutes: 20


### Participants' data (Experiment 2)

In [53]:
# Import dataset from file
participants_data_exp2 = pd.read_csv("../experiment_2_francisco/analysis/data/2018-april-30-export_responses.csv",
                        index_col = 0)

# Create ID column = "pid_plotid"
participants_data_exp2['id'] = participants_data_exp2['pid'].map(str) + '_' + participants_data_exp2['shown_description_plot_id'].map(str)

results = compute_gps_for_dataset(participants_data_exp2)

0_11


NameError: name 'compute_gps' is not defined

In [112]:
save_results(results = results, filename = 'results_list_exp2')

# Compute Gaussian Processes

In [24]:
def debug_filtering(dataset):
    #dataset = dataset[dataset['f0'] == 11]
    #dataset = dataset[dataset['f0'] < 3 ]
    
    return dataset

In [25]:
def scenario(dataset, scenario):
    return dataset[dataset['f2'] == scenario]

In [26]:
np.unique(data_prior['f2'])

array([b'"FB Friends"', b'"Gym members"', b'"Rain"', b'"Salary"',
       b'"Sales"', b'"Temperature"'], 
      dtype='|S13')

### Prior condition

In [ ]:
dataset = debug_filtering(data_prior)
results_prior = compute_gps_for_dataset(dataset)
save_results(results_prior, 'minus-mean-treatment/results_prior', True)

1
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
2
Adding White Kernel to l*p
First Compute OK
LMLs OK
Predictions OK
3
Adding White Kernel to p
Adding White Kernel to l*r
Adding White Kernel to l*p+r
Exception caught computing l*p+r
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
4
Adding White Kernel to l+r
Adding White Kernel to p*r
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
5
Adding White Kernel to l*p
First Compute OK
LMLs OK
Predictions OK
6
Adding White Kernel to l*r
Adding White Kernel to l*p
Adding White Kernel to l*r+p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
7
Adding White Kernel to l*p
Adding White Kernel to p*r
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
8
Adding White Kernel to l+r
Adding White Kernel to p*r
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
9
Adding White Kernel to l+r
Adding White Ke

LMLs OK
Predictions OK
56
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l*p
Adding White Kernel to l+r*p
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
57
Adding White Kernel to l*r
Adding White Kernel to p*r
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
58
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
59
Adding White Kernel to l*p
Exception caught computing l*p
First Compute OK
LMLs OK
Predictions OK
60
Adding White Kernel to l*r
Exception caught computing l*r
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
61
Adding White Kernel to p*r
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
62
Adding White Kernel to l*r
Adding White Kernel to

Adding White Kernel to l*r+p
Exception caught computing l*r+p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
106
Adding White Kernel to l+r
Adding White Kernel to l+p
Adding White Kernel to p*r
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
107
Adding White Kernel to l*p
Exception caught computing l*p
First Compute OK
LMLs OK
Predictions OK
108
Adding White Kernel to p*r
Adding White Kernel to l+r+p
Adding White Kernel to l+r*p
Exception caught computing l+r*p
Adding White Kernel to l*p+r
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
109
Adding White Kernel to l+r
Adding White Kernel to l+p
Adding White Kernel to l*r
Adding White Kernel to l+r*p
Adding White Kernel to l*r+p
First Compute OK
LMLs OK
Predictions OK
110
Adding White Kernel to p+r
Adding White Kernel to l*p
Adding White Kernel to l*p+r
Exception caught computing l*p+r
First Compute OK
LMLs OK
Pre

Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l+r+p
Adding White Kernel to l*r+p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
157
Adding White Kernel to p+r
Adding White Kernel to l+r+p
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
158
First Compute OK
LMLs OK
Predictions OK
159
Adding White Kernel to l+r+p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
160
First Compute OK
LMLs OK
Predictions OK
161
Adding White Kernel to l+r
Adding White Kernel to l*p
Exception caught computing l*p
Adding White Kernel to p*r
Adding White Kernel to l+r*p
Exception caught computing l+r*p
Adding White Kernel to l*r+p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
162
Adding White Kernel to l*r
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
163
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l+r*p
First Co

Adding White Kernel to l*r
First Compute OK
LMLs OK
Predictions OK
215
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
216
Adding White Kernel to l+p
Adding White Kernel to l*r
Exception caught computing l*r
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
217
Adding White Kernel to l*p
Adding White Kernel to l+r+p
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
218
Adding White Kernel to l+r
Adding White Kernel to p+r
Adding White Kernel to l*r
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
219
Adding White Kernel to p*r
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
220
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
221
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
222
Adding White Kernel to l

...

Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
496
Adding White Kernel to p+r
Adding White Kernel to l*p
First Compute OK
LMLs OK
Predictions OK
497
Adding White Kernel to p*r
Exception caught computing p*r
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
498
Adding White Kernel to l*r
Exception caught computing l*r
Adding White Kernel to l+r+p
Adding White Kernel to l*r+p
Adding White Kernel to l*p+r
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
499
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
500
Adding White Kernel to l+r+p
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
501
Adding White Kernel to p+r
Adding White Kernel to l*r
Adding White Kernel to l*r+p
First Compute OK
LMLs OK
Predictions OK
502
Adding White Kernel to l+r
Adding White Kernel to l+r*p
Ad

Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l+r+p
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
548
Adding White Kernel to l*p+r
Exception caught computing l*p+r
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
549
Adding White Kernel to l*r
First Compute OK
LMLs OK
Predictions OK
550
Adding White Kernel to l+r
Adding White Kernel to l*p
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
Exception caught computing l*p+r
First Compute OK
LMLs OK
Predictions OK
551
First Compute OK
LMLs OK
Predictions OK
552
Adding White Kernel to l*r
Adding White Kernel to l*p
Exception caught computing l*p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
553
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l*p
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
554
Adding White Kernel to l*r*p
Exception caught computing l*r*p
Fi

First Compute OK
LMLs OK
Predictions OK
606
Adding White Kernel to l*r
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
607
Adding White Kernel to l*r
Adding White Kernel to l*p
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
608
Adding White Kernel to l*r
Adding White Kernel to l*r+p
First Compute OK
LMLs OK
Predictions OK
609
Adding White Kernel to l*p
First Compute OK
LMLs OK
Predictions OK
610
Adding White Kernel to l+r
First Compute OK
LMLs OK
Predictions OK
611
Adding White Kernel to l*p
Adding White Kernel to p*r
Exception caught computing p*r
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
612
Adding White Kernel to l+r
Adding White Kernel to l*r
Exception caught computing l*r
Adding White Kernel to l*p
Adding White Kernel to l+r+p
Adding White Kernel to l*r+p
Adding White Kernel to l*p+r
Adding White Kernel to l*r*p
Exception 

Adding White Kernel to p*r
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
661
Adding White Kernel to l*r
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
662
Adding White Kernel to l+r+p
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
663
Adding White Kernel to l+r+p
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
664
First Compute OK
LMLs OK
Predictions OK
665
Adding White Kernel to p+r
Adding White Kernel to l*r
Adding White Kernel to p*r
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
666
Adding White Kernel to l+r
Adding White Kernel to p+r
Adding White Kernel to l*r
Exception caught computing l*r
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
667
Adding White Kernel to p*r
First Compute OK
LMLs OK
Predictions OK
668
Adding White Kernel to l+r+p
First Compute OK
LMLs 

Adding White Kernel to l*r+p
Adding White Kernel to l*p+r
Exception caught computing l*p+r
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
Minutes: 667


#### Cross-validation

In [ ]:
if(FALSE):
    #Data required to know the length of the user's dataset
    aux = dataset[dataset['f0'] == 1]
    x = aux['f3']
    y = aux['f4']


    # Dataset to be used
    dataset = debug_filtering(data_prior)

    # Seeding
    random.seed(2016971931)

    # Cross-validation for-loop
    for index in range(0, 10):
        # Copy the dataset
        dataset_to_analyze = copy.deepcopy(dataset)

        # Get the elements to remove
        to_remove = random.sample(list(x), int(len(x) * 0.10)) #10% are removed

        print(to_remove)

        # Remove the elements
        for element in to_remove:
            dataset_to_analyze = dataset_to_analyze[dataset_to_analyze['f3'] != element]

        # Calculate and 
        results_prior = compute_gps_for_dataset(dataset_to_analyze)
        save_results(results_prior, 'cross-validation-prior/results_prior_' + str(index), True)

    save_results(results_prior, 'cross-validation-prior/results_prior_' + str(index), True)

### Posterior condition (only evidence)

#### Full Bayesian posterior condition
This requires calculating the best fitting Kernel in the Prior condition first and using its hyperparameters as the priors of the optimization on the posterior data.

In [27]:
#Datasets
dataset_prior = debug_filtering(data_prior)
dataset_posterior = debug_filtering(data_posterior)

# Only the evidence
dataset_posterior = dataset_posterior[dataset_posterior['f3'] < (365-31+1)]

# Calculate the Full Bayesian results for the posterior
results_posterior = compute_gps_for_dataset(dataset_prior, full_bayesian = True, dataset_posterior = dataset_posterior)

save_results(results_posterior, filename = 'minus-mean-treatment/results_posterior', new_format = True)

1
Adding White Kernel to l+r*p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
2
Adding White Kernel to l*p
First Compute OK
Adding White Kernel to l*r+p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
3
Adding White Kernel to p
Adding White Kernel to l*r
Adding White Kernel to l*p+r
Exception caught computing l*p+r
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
Adding White Kernel to p*r
Full-Bayesian Compute OK
LMLs OK
Predictions OK
4
Adding White Kernel to l+r
Adding White Kernel to p*r
Adding White Kernel to l+r*p
First Compute OK
Adding White Kernel to l+r+p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
5
Adding White Kernel to l*p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
6
Adding White Kernel to l*r
Adding White Kernel to l*p
Adding White Kernel to l*r+p
Adding White Kernel to l*p+r
First Compute OK
Adding White Kernel to l+r*p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
7
Adding White Kernel to l*p
A

Full-Bayesian Compute OK
LMLs OK
Predictions OK
29
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
30
Adding White Kernel to l*r
Exception caught computing l*r
Adding White Kernel to l*p
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
Adding White Kernel to l+r*p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
31
Adding White Kernel to l+r
Adding White Kernel to l+p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
32
Adding White Kernel to l*r
Adding White Kernel to l+r+p
Adding White Kernel to l*r*p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
33
Adding White Kernel to l*p
Adding White Kernel to l*r+p
Adding White Kernel to l*p+r
First Compute OK
Adding White Kernel to l+p
Adding White Kernel to p*r
Full-Bayesian Compute OK
LMLs OK
Predictions OK
34
Adding White Kernel to l+r
Adding White Kernel to l+r*p
First Compute OK
Adding White Kernel to l*p
Full-Bayesian Compute OK
L

LMLs OK
Predictions OK
56
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l*p
Adding White Kernel to l+r*p
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
Adding White Kernel to p+r
Full-Bayesian Compute OK
LMLs OK
Predictions OK
57
Adding White Kernel to l*r
Adding White Kernel to p*r
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
Adding White Kernel to l+r*p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
58
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
59
Adding White Kernel to l*p
Exception caught computing l*p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
60
Adding White Kernel to l*r
Exception caught computing l*r
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
Adding White Kerne

Adding White Kernel to l*r*p
Exception caught computing l*r*p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
80
Adding White Kernel to l+p
Adding White Kernel to l*p
Adding White Kernel to p*r
Adding White Kernel to l+r+p
Adding White Kernel to l+r*p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
81
Adding White Kernel to l*r
Adding White Kernel to l+r+p
Exception caught computing l+r+p
Adding White Kernel to l*r*p
First Compute OK
Adding White Kernel to p*r
Adding White Kernel to l+r*p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
82
Adding White Kernel to l+r
Adding White Kernel to l*p
Adding White Kernel to p*r
Adding White Kernel to l*p+r
First Compute OK
Adding White Kernel to l+r*p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
83
Adding White Kernel to l+p
Adding White Kernel to l*p
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
84
Adding White Kernel to l*r
Adding White Kernel to p

Full-Bayesian Compute OK
LMLs OK
Predictions OK
104
Adding White Kernel to l*p
Adding White Kernel to p*r
Adding White Kernel to l*p+r
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
105
Adding White Kernel to p
Adding White Kernel to l+r
Adding White Kernel to l*r
Exception caught computing l*r
Adding White Kernel to l*p
Adding White Kernel to l*r+p
Exception caught computing l*r+p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
Adding White Kernel to l+r+p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
106
Adding White Kernel to l+r
Adding White Kernel to l+p
Adding White Kernel to p*r
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
First Compute OK
Adding White Kernel to l+r*p
Exception caught computing l+r*p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
107
Adding White Kernel to l*p
Exception caught computing l*p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
108
Adding White Kernel to p*r
Adding White 

...

First Compute OK
Adding White Kernel to p*r
Adding White Kernel to l*r+p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
606
Adding White Kernel to l*r
Adding White Kernel to l*r*p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
607
Adding White Kernel to l*r
Adding White Kernel to l*p
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
608
Adding White Kernel to l*r
Adding White Kernel to l*r+p
First Compute OK
Adding White Kernel to l*r*p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
609
Adding White Kernel to l*p
First Compute OK
Adding White Kernel to l*r+p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
610
Adding White Kernel to l+r
First Compute OK
Adding White Kernel to l*r+p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
611
Adding White Kernel to l*p
Adding White Kernel to p*r
Exception caught computing p*r
Adding White Kernel to l+r*p
Adding White Kerne

Adding White Kernel to l*r
Adding White Kernel to l+r*p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
632
Adding White Kernel to l+p
Adding White Kernel to l*r
Adding White Kernel to l*p
Adding White Kernel to l+r+p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
633
Adding White Kernel to l+r
Adding White Kernel to l+p
First Compute OK
Adding White Kernel to l*p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
634
Adding White Kernel to l+p
Adding White Kernel to p+r
Adding White Kernel to l*r
Adding White Kernel to l*p
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
Adding White Kernel to l*r*p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
635
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l*p
Adding White Kernel to l+r+p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
636
Adding White Kernel to p
Exception caught computing p
Adding White Kernel to l+r
Adding White Kernel to

Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
658
Adding White Kernel to l*r
Adding White Kernel to l*p
First Compute OK
Adding White Kernel to l*r*p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
659
Adding White Kernel to l+p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
660
Adding White Kernel to l+p
Adding White Kernel to l*r
Exception caught computing l*r
Adding White Kernel to l*p
Adding White Kernel to p*r
Adding White Kernel to l*r*p
First Compute OK
Adding White Kernel to l+r*p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
661
Adding White Kernel to l*r
Adding White Kernel to l*r*p
First Compute OK
Adding White Kernel to l+r*p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
662
Adding White Kernel to l+r+p
Adding White Kernel to l+r*p
First Compute OK
Adding White Kernel to l*r+p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
663
Adding White Kernel to l+r+p
Adding White Ker

Adding White Kernel to l*r
Adding White Kernel to l*p
Adding White Kernel to l*p+r
Adding White Kernel to l*r*p
First Compute OK
Adding White Kernel to l*p+r
Exception caught computing l*p+r
Full-Bayesian Compute OK
LMLs OK
Predictions OK
685
Adding White Kernel to l*r
Adding White Kernel to l+r+p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
686
Adding White Kernel to l*r
Adding White Kernel to l+r+p
Adding White Kernel to l*p+r
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
687
Adding White Kernel to l*r+p
Exception caught computing l*r+p
Adding White Kernel to l*p+r
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
688
Adding White Kernel to l*r
Adding White Kernel to l*r+p
First Compute OK
Adding White Kernel to l*p
Full-Bayesian Compute OK
LMLs OK
Predictions OK
689
Adding White Kernel to l+r+p
Adding White 

First Compute OK
Full-Bayesian Compute OK
LMLs OK
Predictions OK
712
Adding White Kernel to l+r
Adding White Kernel to l+p
Adding White Kernel to p+r
Exception caught computing p+r
Adding White Kernel to l*r
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
Exception caught computing l*p+r
First Compute OK
Adding White Kernel to p*r
Exception caught computing p*r
Full-Bayesian Compute OK
LMLs OK
Predictions OK
713
Adding White Kernel to l+r
Adding White Kernel to l+p
Adding White Kernel to l*r
Adding White Kernel to l*p
Adding White Kernel to p*r
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
Exception caught computing l*p+r
First Compute OK
Adding White Kernel to p*r
Exception caught computing p*r
Full-Bayesian Compute OK
LMLs OK
Predictions OK
714
Adding White Kernel to l*r
Adding White Kernel to l*r+p
Adding White Kernel to l*p+r
Exception caught computing l*p+r
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
Full-Bayesian Compute OK

### Posterior condition (participants' data)

In [27]:
#Datasets
dataset_posterior = debug_filtering(data_posterior)

# Only the evidence
dataset_posterior = dataset_posterior[dataset_posterior['f3'] >= (365-31+1)]

# Calculate the Full Bayesian results for the posterior
results_posterior = compute_gps_for_dataset(dataset_posterior, full_bayesian = False)

save_results(results_posterior, filename = 'minus-mean-treatment/results_posterior_participants', new_format = True)

1
Adding White Kernel to p*r
First Compute OK
LMLs OK
Predictions OK
2
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l*p+r
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
3
Adding White Kernel to l+r+p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
4
Adding White Kernel to l*p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
5
Adding White Kernel to l*p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
6
Adding White Kernel to l*p
Adding White Kernel to l*p+r
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
7
Adding White Kernel to p*r
Adding White Kernel to l+r+p
First Compute OK
LMLs OK
Predictions OK
8
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
9
Adding White Kernel to l*p
First Compute OK
LMLs OK
Predictions OK
10
Adding White Kernel to l*p
Fir

First Compute OK
LMLs OK
Predictions OK
71
Adding White Kernel to p
Adding White Kernel to p*r
Adding White Kernel to l*r+p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
72
Adding White Kernel to p+r
Adding White Kernel to l*r
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
73
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
74
Adding White Kernel to l*r
First Compute OK
LMLs OK
Predictions OK
75
Adding White Kernel to l+r
Adding White Kernel to p+r
Exception caught computing p+r
Adding White Kernel to l*p
Adding White Kernel to l+r+p
First Compute OK
LMLs OK
Predictions OK
76
Adding White Kernel to l*r
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
77
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l+r+p
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
78
Adding White Kernel to l*p
Adding White 

Adding White Kernel to l+r+p
First Compute OK
LMLs OK
Predictions OK
140
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
141
Adding White Kernel to l+r
Adding White Kernel to l+r+p
First Compute OK
LMLs OK
Predictions OK
142
Adding White Kernel to p+r
Adding White Kernel to p*r
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
143
Adding White Kernel to l+r
First Compute OK
LMLs OK
Predictions OK
144
Adding White Kernel to l*r
Adding White Kernel to l+r+p
Adding White Kernel to l*p+r
Exception caught computing l*p+r
First Compute OK
LMLs OK
Predictions OK
145
First Compute OK
LMLs OK
Predictions OK
146
Adding White Kernel to p+r
Adding White Kernel to l*r
Adding White Kernel to l+r*p
Adding White Kernel to l*r+p
First Compute OK
LMLs OK
Predictions OK
147
Adding White Kernel to l
Adding White Kernel to l*p
First Compute OK
LMLs OK
Predictions OK
148
Adding White Kernel to l*p
Adding White Kern

Predictions OK
216
Adding White Kernel to l*r
Adding White Kernel to l*p
First Compute OK
LMLs OK
Predictions OK
217
First Compute OK
LMLs OK
Predictions OK
218
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
219
Adding White Kernel to p+r
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
220
Adding White Kernel to l+p
Adding White Kernel to l+r+p
Adding White Kernel to l*r+p
First Compute OK
LMLs OK
Predictions OK
221
Adding White Kernel to l+r
Adding White Kernel to l*p
First Compute OK
LMLs OK
Predictions OK
222
Adding White Kernel to p+r
First Compute OK
LMLs OK
Predictions OK
223
First Compute OK
LMLs OK
Predictions OK
224
Adding White Kernel to l+r
Adding White Kernel to l+p
Adding White Kernel to p+r
Adding White Kernel to l*r
Adding White Kernel to l*p
Adding White Kernel to l+r+p
Adding White Kernel to l*r*p
First Compute OK
LM

First Compute OK
LMLs OK
Predictions OK
302
Adding White Kernel to l+r*p
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
303
First Compute OK
LMLs OK
Predictions OK
304
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l+r*p
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
305
Adding White Kernel to l+r+p
First Compute OK
LMLs OK
Predictions OK
306
Adding White Kernel to p+r
First Compute OK
LMLs OK
Predictions OK
307
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
308
Adding White Kernel to l+r*p
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
309
Adding White Kernel to p+r
Adding White Kernel to l*p
Exception caught computing l*p
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
310
Adding White Kernel to l+p
Exception caught computing l+p
First Compute OK
LMLs OK

Adding White Kernel to l+r
Adding White Kernel to l+r+p
First Compute OK
LMLs OK
Predictions OK
379
Adding White Kernel to l+p
First Compute OK
LMLs OK
Predictions OK
380
Adding White Kernel to l+r+p
First Compute OK
LMLs OK
Predictions OK
381
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
382
Adding White Kernel to l+p
Adding White Kernel to l*r
Exception caught computing l*r
Adding White Kernel to l*p+r
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
383
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
384
Adding White Kernel to l*r
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
385
Adding White Kernel to p*r
Adding White Kernel to l+r+p
Adding White Kernel to l*p+r
Exception caught computing l*p+r
First Compute OK
LMLs OK
Predictions OK
386
Adding White Kernel to l+r
First Compute OK
LMLs OK
Predictions OK
387

LMLs OK
Predictions OK
454
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l+r+p
Adding White Kernel to l+r*p
Exception caught computing l+r*p
First Compute OK
LMLs OK
Predictions OK
455
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
456
Adding White Kernel to p
First Compute OK
LMLs OK
Predictions OK
457
Adding White Kernel to p+r
First Compute OK
LMLs OK
Predictions OK
458
Adding White Kernel to l*p
Adding White Kernel to l+r+p
Adding White Kernel to l*p+r
Exception caught computing l*p+r
First Compute OK
LMLs OK
Predictions OK
459
Adding White Kernel to l+r
Adding White Kernel to l+r+p
First Compute OK
LMLs OK
Predictions OK
460
Adding White Kernel to p*r
First Compute OK
LMLs OK
Predictions OK
461
Adding White Kernel to p
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
462
Adding White Kernel to l*p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predi

First Compute OK
LMLs OK
Predictions OK
532
Adding White Kernel to l*p
Adding White Kernel to p*r
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
533
Adding White Kernel to l+p
Adding White Kernel to l*p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
534
Adding White Kernel to p+r
Adding White Kernel to l*p
First Compute OK
LMLs OK
Predictions OK
535
Adding White Kernel to l*r
Adding White Kernel to p*r
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
536
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
537
Adding White Kernel to l*r+p
First Compute OK
LMLs OK
Predictions OK
538
Adding White Kernel to l+r
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
539
Adding White Kernel to p+r
First Compute OK
LMLs OK
Predi

Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
608
Adding White Kernel to l+r
Adding White Kernel to l*r
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
609
Adding White Kernel to l+r
First Compute OK
LMLs OK
Predictions OK
610
Adding White Kernel to l+r
First Compute OK
LMLs OK
Predictions OK
611
Adding White Kernel to l*r+p
First Compute OK
LMLs OK
Predictions OK
612
Adding White Kernel to p+r
Adding White Kernel to l*p
Adding White Kernel to l+r+p
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
613
Adding White Kernel to l*p
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
614
Adding White Kernel to l+r
Adding White Kernel to l+r*p
Adding White Kernel to l*p+r
Adding White Kernel to l*r*p
Exception caught computing l*r*p
First Compute OK
LMLs OK
Predictions OK
615
Adding White Kernel to l*r
Adding White Kernel to l*p
Exception caught computing l*p
Adding White Kernel to l+r+p
Adding White Kernel to l

Predictions OK
681
Adding White Kernel to l*p
Adding White Kernel to l*p+r
Exception caught computing l*p+r
First Compute OK
LMLs OK
Predictions OK
682
Adding White Kernel to l*p
Adding White Kernel to p*r
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
683
Adding White Kernel to l*r
First Compute OK
LMLs OK
Predictions OK
684
Adding White Kernel to l*r
Adding White Kernel to l+r*p
Adding White Kernel to l*r+p
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
685
Adding White Kernel to l+r
Adding White Kernel to l+r*p
First Compute OK
LMLs OK
Predictions OK
686
Adding White Kernel to p*r
Adding White Kernel to l*r*p
First Compute OK
LMLs OK
Predictions OK
687
Adding White Kernel to l+r
Adding White Kernel to l+r+p
Adding White Kernel to l*p+r
First Compute OK
LMLs OK
Predictions OK
688
Adding White Kernel to l*r
First Compute OK
LMLs OK
Predictions OK
689
First Compute OK
LMLs OK
Predictions OK
690
Adding White Kernel to l*p
Adding White Kernel t

# Plots

In [ ]:
plot_predictions(results_posterior, data_posterior, 8, 'l+p')

In [ ]:
plot_predictions(results_prior, data_prior, 9, 'l+r*p')

# Results analysis

R libraries

In [ ]:
%%R
source("tools.R")

### Import the results

In [ ]:
with open('output/results_prior.json', 'r') as fp:
    results_prior = json.load(fp)

In [ ]:
with open('output/results_prior_maxs.json', 'r') as fp:
    results_prior_maxs = json.load(fp)

In [ ]:
with open('output/results_posterior.json', 'r') as fp:
    results_posterior = json.load(fp)

In [ ]:
#aux = pd.DataFrame(results_posterior['maxs']).T
#aux.columns = ['kernel', 'lml']

#for i, row in aux.iterrows():
#    aux.set_value(i, 'kernel', row['kernel'][0])

#aux.to_csv('output/results_posterior_maxs.csv', sep=',', encoding='utf-8')

In [ ]:
#aux = pd.DataFrame(results_prior['lmls']).T
#aux.to_csv('output/lmls_prior.csv', sep=',', encoding='utf-8')

aux = pd.DataFrame(results_posterior['lmls']).T
aux.to_csv('output/lmls_posterior.csv', sep=',', encoding='utf-8')

### Functions to analyze data

In [ ]:
%%R
kernels <- c("l", "p", "r", "l+p", "l+r", "p+r", "l*r", "l*p", "p*r", "l+r+p", "l+r*p", "l*r+p", "l*p+r", "l*r*p")

get_proportions <- function(res_x) {
    total <- length(res_x[1][[1]])

    props <- res_x %>% 
                group_by(kernel) %>%
                summarize(proportion = length(kernel)/total,
                          lo_ci = prop.test(x=length(kernel), n=total, conf.level=0.95, correct = FALSE)$conf[1],
                          hi_ci = prop.test(x=length(kernel), n=total, conf.level=0.95, correct = FALSE)$conf[2])
    
    props$kernel <- factor(props$kernel, levels=kernels)
    
    # Adding zeros
    for (k in kernels){
        # If the kernel is NOT present
        if (sum(props$kernel == k) == 0){         
            new_row <- c(k, 0, 0, 0)
            
            props <- rbind(props, new_row)
        } 
    }
    
    # Reorder factors
    #props$kernel <- factor(props$kernel, levels=c("l", "p", "r", "l+p", "l+r", "p+r", "l*r", "l*p", "p*r", "l+r+p", "l+r*p", "l*r+p", "l*p+r", "l*r*p"))
    
    props$proportion <- as.numeric(props$proportion)
    
    return (props)
}

In [ ]:
%%R

plot_proportions <- function(prop_data, title, hide_x=FALSE, hide_y=FALSE) {
    plot <- prop_data %>%
                ggplot(aes(x=kernel, y=proportion)) + 
                    geom_bar(stat="identity") +
                    #ylim(0, 0.5) +
                    coord_cartesian (ylim=c(0,0.5)) +
                    labs(title = title) +
                    ggthemes::theme_few() +
                    xlab("Kernel composition") + ylab("Proportion") +
                    geom_errorbar(aes(ymin=as.numeric(lo_ci), ymax=as.numeric(hi_ci)),
                                      width=.3,                   
                                      position=position_dodge(.9)) +
                    theme(axis.text.x = element_text(angle = 90,  vjust = 0.5, hjust=0),
                          text = element_text(size=12, family="serif"),
                          plot.title = element_text(hjust = 0.5))
    
    if(hide_x){
        plot <- plot + theme(axis.title.x=element_blank())
    }
    if(hide_y){
        plot <- plot + theme(axis.title.y=element_blank())
    }
    
    return(plot)
}

### Proportions of best fitting kernel composition in the Prior condition, per scenario

In [ ]:
maxs_prior = pd.DataFrame(data=results_prior['maxs']).T
#maxs_prior = pd.DataFrame(data=results_posterior['maxs']).T
maxs_prior.columns = ['kernel', 'lml']

for i in maxs_prior.index:
    maxs_prior['kernel'][i] = maxs_prior['kernel'][i][0]
    
maxs_prior['id'] = maxs_prior.index.values.tolist()

maxs_prior_i = maxs_prior['id'].values.tolist()
maxs_prior_k = maxs_prior['kernel'].values.tolist()
maxs_prior_l = maxs_prior['lml'].values.tolist()

#maxs_posterior =  = pd.Series(results_posterior['maxs'])

Data prior

In [ ]:
%%R 
data_prior <- read_csv("data/for_composititional_analysis_prior.csv")

dict_prior <- data_prior %>% 
                        group_by(id, scenario) %>%
                        summarize()

In [ ]:
%%R -i maxs_prior_i,maxs_prior_k,maxs_prior_l

maxs_prior <- data.frame(id=maxs_prior_i,kernel=maxs_prior_k,lml=maxs_prior_l)

maxs_prior <- merge(x = maxs_prior, y = dict_prior, by = c("id", "id"), all.x = TRUE)

In [ ]:
%%R
maxs_prior_temperature <- maxs_prior %>% filter(scenario == "Temperature")
maxs_prior_rain <- maxs_prior %>% filter(scenario == "Rain")
maxs_prior_sales <- maxs_prior %>% filter(scenario == "Sales")
maxs_prior_gym <- maxs_prior %>% filter(scenario == "Gym members")
maxs_prior_salary <- maxs_prior %>% filter(scenario == "Salary")
maxs_prior_fb <- maxs_prior %>% filter(scenario == "FB Friends")

prop_temperature <- get_proportions(maxs_prior_temperature)
prop_rain <- get_proportions(maxs_prior_rain)
prop_sales <- get_proportions(maxs_prior_sales)
prop_gym <- get_proportions(maxs_prior_gym)
prop_salary <- get_proportions(maxs_prior_salary)
prop_fb <- get_proportions(maxs_prior_fb)

In [ ]:
%%R
p1 <- plot_proportions(prop_temperature, "Temperature", hide_x=TRUE)
p2 <- plot_proportions(prop_rain, "Rain", hide_x=TRUE)
p3 <- plot_proportions(prop_sales, "Sales", hide_x=TRUE, hide_y=TRUE)
p4 <- plot_proportions(prop_gym, "Gym members", hide_y=TRUE, hide_x=TRUE)
p5 <- plot_proportions(prop_salary, "Salary", hide_x=TRUE, hide_y=TRUE)
p6 <- plot_proportions(prop_fb, "FB Friends", hide_y=TRUE, hide_x=TRUE)

svg("Images/kernels_prior.svg", width=8, height=4)
multiplot(p1, p2, p3, p4, p5, p6, cols=3)
dev.off()

### Compare the best kernel composition SSE versus those of `l` and `r`

Functions

In [ ]:
def get_posterior_curve(cid):
    cid = int(cid)
    dataset = data_posterior
    
    # Filter the relevant data
    filtered_data = dataset[dataset['f0'] == cid]
    
    # Get X and Y
    x = filtered_data['f3']
    y = filtered_data['f4']
    
    df = pd.DataFrame([x, y]).T
    
    df.columns=["x", "y"]
    
    return df

In [ ]:
def get_posterior_prediction(cid, kernel_name):
    # Get the target values
    x = Xpredictions
    
    y = results_posterior['predictions'][cid][kernel_name]['mean']
    
    y_var = results_posterior['predictions'][cid][kernel_name]['var']
    
    # Squeeze the matrices
    x = np.squeeze(x); y = np.squeeze(y); y_var = np.squeeze(y_var); 
    
    df = pd.DataFrame([x, y, y_var]).T
    
    df.columns=["x", "y", "y_var"]
    
    return df

In [ ]:
def get_y_value(df, x):
    return df[df['x']==x]['y'].tolist()[0]

In [ ]:
def compute_SSE(true_dataframe, prediction_dataframe, minX = 365-31):
    
    df1 = true_dataframe[true_dataframe['x'] > minX]
    df2 = prediction_dataframe[prediction_dataframe['x'] > minX]
    
    sse = 0
    
    for x in df1['x'] :
        error = get_y_value(df1, x) - get_y_value(df2, x)
        sse += (error*error)
        
    # Root mean squared deviation
    rmsd = np.sqrt(sse / len(df1))
    
    # Normalized
    return rmsd / (np.max(df1['y']) - np.min(df1['y']))

End functions

In [ ]:
columns = ["cid", "sse_l", "sse_r", "sse_best"]

sses = pd.DataFrame(columns = columns)

for cid in np.unique(data_prior['f0']):
    
    if cid != 533:
        cid = str(cid)
        
        # Get the best kernel for the curve
        best_kernel_name = results_prior['maxs'][cid][0][0]

        # Gest the predictions of the l, r, and best kernel
        prediction_l = get_posterior_prediction(cid, 'l')
        prediction_r = get_posterior_prediction(cid, 'r')
        prediction_best = get_posterior_prediction(cid, best_kernel_name)

        # Get the actual values in the posterior condition
        prediction = get_posterior_curve(cid)

        # Calculate the Sum of squared errors
        sse_l = compute_SSE(prediction, prediction_l)
        sse_r = compute_SSE(prediction, prediction_r)
        sse_best = compute_SSE(prediction, prediction_best)

        results = pd.DataFrame([cid, sse_l, sse_r, sse_best]).T
        results.columns = ["cid", "sse_l", "sse_r", "sse_best"]

        sses = sses.append(results, ignore_index=True)

In [ ]:
def mean_confidence_interval(data, confidence=0.99):
    a = 1.0*np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

Summarizing

In [ ]:
print('NRMSD    Mean  -CI99  +CI99')

print ('Linear:', np.round(mean_confidence_interval(sses['sse_l']),2))

print ('RBF:   ', np.round(mean_confidence_interval(sses['sse_r']),2))

print ('Best:  ', np.round(mean_confidence_interval(sses['sse_best']),2))

In [ ]:
sse_l = sses['sse_l'].tolist()
sse_r = sses['sse_r'].tolist()
sse_best = sses['sse_best'].tolist()

Paired t-test

In [ ]:
%%R -i sse_l,sse_r,sse_best

t.test(sse_l, sse_r, paired=TRUE, alternative="greater")

In [ ]:
%%R -i sse_l,sse_r,sse_best

t.test(sse_l, sse_best, paired=TRUE, alternative="greater")

In [ ]:
%%R -i sse_l,sse_r,sse_best

t.test(sse_r, sse_best, paired=TRUE)

In [ ]:
%%R -i sse_l,sse_r,sse_best

sse_l = data.frame(kernel="l", sse=sse_l)
sse_r = data.frame(kernel="r", sse=sse_r)
sse_best = data.frame(kernel="best", sse=sse_best)

sses = rbind(sse_l, sse_r, sse_best)

In [ ]:
%%R
p <- ggplot(sses, aes(x=kernel, y=sse, group=kernel)) +
        geom_point(alpha=0.1, aes(size=3)) +
        xlab("Kernel composition") + ylab("NRMSE") +
        theme(text = element_text(size=20, family="serif"),
            plot.title = element_text(hjust = 0.5),
            legend.position="none")

#svg("Images/kernels_nmse.svg")
p
#dev.off()

In [ ]:
%%R
p <- ggplot(sses, aes(x=kernel, y=sse, group=kernel)) +
        geom_point(alpha=0.1, aes(size=3)) +
        xlab("Kernel composition") + ylab("NRMSE") +
        ylim(0,60) + 
        theme(text = element_text(size=20, family="serif"),
            plot.title = element_text(hjust = 0.5),
            legend.position="none")

#svg("Images/kernels_nmse2.svg")
p
#dev.off()

# Judgmental forecasting effects in compositions

In [ ]:
cid =  '1'
best_kernel_name = results_prior_maxs[cid][0][0]
str(best_kernel_name)

In [ ]:
predictions = pd.DataFrame()

for cid in np.unique(data_prior['f0']):
    
    if cid != 533:
        cid = str(cid)

        # Best kernel
        best_kernel_name = str(results_prior_maxs[cid][0][0])
        prediction_best = get_posterior_prediction(cid, best_kernel_name)

        # Get the actual values in the posterior condition
        prediction = get_posterior_curve(cid)
        
        # Filter the Best Kernel predictions; one every 5 days and with a max
        prediction_best = prediction_best[(prediction_best['x'] % 5 == 1) & (prediction_best['x'] <= max(prediction['x']))]
        
        # Adding columns
        prediction_best = prediction_best.assign(cid=cid, kernel_prediction=True)
        
        prediction = prediction.assign(cid=cid, kernel_prediction=False)
        
        # Concatenating to predictions dataframe
        predictions = predictions.append(prediction_best, ignore_index=True)
        predictions = predictions.append(prediction, ignore_index=True)

In [ ]:
%%R

data_posterior <- read_csv("data/for_composititional_analysis_posterior.csv")

dict_posterior <- data_posterior %>% 
                        group_by(id, scenario, condition) %>%
                        summarize()

In [ ]:
%%R -i predictions

colnames(predictions) <- c("id", "kernel_prediction", "day", "value", "var")
predictions <- merge(x = predictions, y = dict_posterior, by = c("id"), all.x = TRUE)

In [ ]:
%%R
predictions %>% head

write.csv(predictions, 'output/posterior_predictions.csv')

Now that the data is ready, let's graph the Judgmental forecasting phenomena

In [ ]:
%%R
dat <- predictions %>% filter(kernel_prediction)
readable_scenarios <- c("Temperature", "Rain", "Sales", "Gym members", "Salary", "FB Friends")
condition_names = c("Prior", "Posterior-Positive", "Posterior-Stable", "Posterior-Negative")

# Order on the plot
dat$condition <- factor( dat$condition, 
                         levels = condition_names)

dat$scenario <- factor( dat$scenario, 
                        levels = readable_scenarios)

### NORM DATA

In [ ]:
%%R

df <- dat %>%
        group_by(condition, scenario, day) %>%
        summarize(mean(value))

df$slope_scale <- mapvalues(df$condition,
                              from = c("Prior", "Posterior-Positive", "Posterior-Stable", "Posterior-Negative"),
                              to   = c(0, 1, 0, -1))

df$range <- mapvalues(df$scenario,
                              from = c("Temperature", "Rain", "Sales", "Gym members", "Salary", "FB Friends"),
                              to   = c( 40--10,        100-0,  5000-0,  50-0,          50-0,     1000-0))

df$y_intercept <- mapvalues(df$scenario,
                              from = c("Temperature", "Rain", "Sales", "Gym members", "Salary", "FB Friends"),
                              to   = c( 15,            30,     2500,    25,            20,       500))


# Transformations to numbers
df$slope_scale <- as.numeric(levels(df$slope_scale))[df$slope_scale]
df$range <- as.numeric(levels(df$range))[df$range]
df$y_intercept <- as.numeric(levels(df$y_intercept))[df$y_intercept]

# Calculations
df$last_point <- df$y_intercept + 0.05 * df$range * df$slope_scale * 4 

df$slope <- (df$last_point - df$y_intercept) / ((365-31)-1)

df$underlying_trend <- ((df$day-1) * df$slope + df$y_intercept)

scaling <- df %>%
                    group_by(condition, scenario) %>%
                    summarize(y_intercept = unique(y_intercept), last_point= unique(last_point), range = unique(range))

# Use the scaling object to scale the data
norm_data <- merge(x = dat, y = scaling, by = c("condition", "scenario"), all.x = TRUE)

norm_data <- norm_data %>%
                group_by() %>%
                mutate(value_scaled = (value - y_intercept)/(range))
# Normalize the data
norm_data <- norm_data %>%
                group_by() %>%
                mutate(value_norm = scale(value_scaled))

# Calculate the means and SDs
means <- norm_data %>%
            group_by() %>%
            summarize(value_mean = mean(value_scaled),
                      value_sd = sd(value_scaled))

# Remove the scaled data
norm_data <- norm_data %>% select(-value_scaled)

In [ ]:
%%R

# Scaled trend
underlying_trend <- df %>%
            group_by() %>%
            mutate(underlying_trend_scaled = (underlying_trend - y_intercept)/(range))

# Normalize trend
underlying_trend <- underlying_trend %>%
            mutate(underlying_trend_norm = ((underlying_trend_scaled - means$value_mean)/means$value_sd))

underlying_trend <- underlying_trend %>%
            select(condition, scenario, day, underlying_trend_norm)
    
    
# Merge the trend into the normalized data
norm_data <- merge(x=norm_data, y=underlying_trend, by=c("condition", "scenario", "day"), all.x=TRUE)

# Merging transforms the `value_norm` column into a matrix. This fixes that. No one knows why that happens.
norm_data$value_norm <- c(norm_data$value_norm)

underlying_trend$id <- 1


norm_data <- norm_data %>%
                mutate(abs_error = abs(underlying_trend_norm - value_norm))

## Grid plot function

In [ ]:
%%R

grid_plot <- function(dplot, trend, res=30, type=1) {
    
    if(type == 1){
        y = "value"
        group = "id"
        ylab = "Value"
        trend_y = "value"
    }
    else if(type == 2){
        y = "value_norm"
        group = "id"
        ylab = "Scaled value"
        trend_y = "underlying_trend_norm"
    }
    
    # Subsetting the data
    #dplot <- subset(dplot, day %in% seq(min(dplot$day), max(dplot$day), res))
    
    # Plotting
    p <- dplot %>%
            ggplot(aes_string(x="day", y=y, group=group)) + 
            geom_line(col="steelblue", alpha=0.2) +
            facet_grid(scenario~condition, scales="free_y") +
            theme_classic() + 
            xlab("Year") + 
            ylab(ylab) +
            geom_line(data=trend, aes_string(x="day", y=trend_y), colour="black") +
            scale_x_continuous(breaks=c(0, 365, 365*2, 365*3), labels=c('Y1', 'Y2', 'Y3', 'Y4')) +
            geom_vline(data=filter(trend, condition!="Prior"), aes(xintercept=365-31), colour="steelblue") +
            theme(  text = element_text(size=12, family="serif"),
                    axis.line = element_line(colour="black", size=0.1),
                    panel.spacing.x = unit(0.5, "lines"),
                    panel.spacing.y = unit(0.8, "lines"),
                    panel.background = element_blank(),
                    panel.grid.major = element_blank(), 
                    panel.grid.minor = element_blank(),
                    panel.border = element_rect(colour="black", fill=NA, size=0.1))
        
    return (p)
}

# TREND DAMPING

### Plot: trend damping

In [ ]:
%%R

# Filter out the prior condition and {temperature, rain} scenarios. Also filter out before year 1 and after year 4
filtered_norm_data <- norm_data %>%
        filter(condition != 'Prior', condition != 'Posterior-Stable', scenario != 'Temperature', scenario != 'Rain', day > 365-31, day <= 365*3 )

filtered_underlying_trend <- underlying_trend %>%
        filter(condition != 'Prior', condition != 'Posterior-Stable', scenario != 'Temperature', scenario != 'Rain', day > 365-31, day <= 365*3 )

p <- grid_plot(filtered_norm_data, filtered_underlying_trend, type=2, res=5)

#svg("Images/kernel_trend_damping.svg", width=5, height=5)
p
#dev.off()

In [ ]:
%%R

# Adding `damped` column
norm_data <- norm_data %>%
                mutate(damped = value_norm < underlying_trend_norm) 

norm_data$damped <- ifelse(norm_data$condition == "Posterior-Positive", norm_data$damped, !norm_data$damped)

# Summarize overall scenarios
damping_overall <- norm_data %>% 
                        filter(condition != 'Prior', condition != 'Posterior-Stable', scenario != 'Temperature', scenario != 'Rain', day > 365-31, day <= 365*3 ) %>%
                        group_by(condition, day) %>%
                        summarize(damped_proportion = sum(damped*1)/length(damped),
                                  ci_lower = ifelse(damped_proportion<1, t.test(damped*1, conf.level=0.95)$conf.int[1], damped_proportion),
                                  ci_upper = ifelse(damped_proportion<1, t.test(damped*1, conf.level=0.95)$conf.int[2], damped_proportion)) %>%
                        mutate(group = 1)
                                  
# Summarize per day
damping_detail <- norm_data %>%
                            filter(condition != 'Prior', condition != 'Posterior-Stable', scenario != 'Temperature', scenario != 'Rain', day > 365-31, day <= 365*3 ) %>%
                            group_by(condition, scenario, day) %>%
                            summarize(damped_proportion = sum(damped*1)/length(damped))

In [ ]:
%%R
# Reversing the conditions (design thing)
dplot <- damping_overall
dplot$condition <- factor(dplot$condition, levels = rev(levels(dplot$condition)))

# Plotting
p <- plot_with_ribbon(dplot, type=1)

#svg("Images/kernel_trend_damping_overall_with_ci.svg", width=6, height=2)
p
#dev.off()

In [ ]:
%%R
dplot <- norm_data %>% 
            filter(condition != 'Prior', condition != 'Posterior-Stable', scenario != 'Temperature', scenario != 'Rain', day > 365-31, day <= 365*3 ) %>%
            group_by(condition, day) %>%
            summarize(mae = mean(abs_error),
                      ci_lower = t.test(abs_error, conf.level=0.999)$conf.int[1],
                      ci_upper = t.test(abs_error, conf.level=0.999)$conf.int[2]) %>%
            mutate(group = 1)

# Reversing the conditions (design thing)
dplot$condition <- factor(dplot$condition, levels = rev(levels(dplot$condition)))

# Plotting
p <- plot_with_ribbon(dplot, type=2)

#svg("Images/kernel_trend_damping_overall_mae_with_ci.svg", width=6, height=2)
p
#dev.off()

In [ ]:
%%R
ggsave(file="Images/kernel_trend_damping_overall_mae_with_ci.svg", plot=p, width=10, height=8)

In [ ]:
%%R
dplot <- norm_data %>% 
            filter(condition != 'Prior', condition != 'Posterior-Stable', scenario != 'Temperature', scenario != 'Rain', day > 365-31, day <= 365*3 ) %>%
            group_by(condition, scenario, day) %>%
            summarize(mae = mean(abs_error),
                      ci_lower = t.test(abs_error, conf.level=0.95)$conf.int[1],
                      ci_upper = t.test(abs_error, conf.level=0.95)$conf.int[2]) %>%
            mutate(group = 1)

# Reversing the conditions (design thing)
dplot$condition <- factor(dplot$condition, levels = rev(levels(dplot$condition)))

p1 <- plot_with_ribbon(dplot %>% filter(scenario=="Sales"),       type=3,  hide_legend=TRUE, ylab="Mean absolute error")
p2 <- plot_with_ribbon(dplot %>% filter(scenario=="Gym members"), type=3,  hide_legend=TRUE)
p3 <- plot_with_ribbon(dplot %>% filter(scenario=="Salary"),      type=3,  hide_legend=TRUE)
p4 <- plot_with_ribbon(dplot %>% filter(scenario=="FB Friends"),  type=3,  hide_legend=TRUE)

#svg("Images/kernels_trend_damping_scenarios_mae_with_ci.svg", width=6, height=1.5)
#multiplot(p1, p3, p2, p4, cols=4)
#dev.off()

# DEBUG

In [ ]:
%%R

plot_with_ribbon <- function(dplot, type, title="", xlab="Year", ylab="", hide_legend=FALSE, group="condition", legend_title="Condition") {
    
    if(type == 1){
        y = "damped_proportion"
        ylab = "Proportion of\ndamped predictions"
    }
    else if(type == 2){
        y = "mae"
        ylab = "Mean absolute error"
    }
    else if(type == 3){
        y = "mae"
        title = dplot$scenario[1]
    }
    else if(type == 4){
        y = "mae"
        legend_title = "Noise Group"
        group = "noise"
        title = dplot$condition[1]
    }
    else if(type==5 || type==6){
        y = "mae"
        legend_title = "Noise Group"
        group = "noise"
    }
    
    p <- ggplot(data=dplot, aes_string(x="day", y=y, group=group, colour=group)) +
            geom_line(size=1) +
            geom_ribbon(aes(ymin=ci_lower, ymax=ci_upper), alpha=0.2) +
            #scale_x_continuous(limits +
            #coord_cartesian(ylim = c(0.65, 1.00), xlim = c(365-31, 365*2-10)) +
            labs(colour = legend_title) +
            xlab(xlab) + ylab(ylab) + ggtitle(title) +
            scale_x_continuous(breaks=c(0, 365, 365*2, 365*3), labels=c('Y1', 'Y2', 'Y3', 'Y4')) +
            theme_bw() + 
            #ggthemes::theme_few() + 
            theme(text = element_text(size=12, family="serif")) +
            #theme(legend.position=c(0.82, 0.40)) +
            theme(plot.title = element_text(hjust = 0.5)) #+
            #ggthemes::scale_color_solarized()
    
        
    if(type == 3){
        p <- p + coord_cartesian(ylim = c(0, 3))
    }
    else if(type == 4) {
        p <- p + theme(legend.position=c(1.0, 0.4))
    }
    else if(type == 5) {
        p <- p + coord_cartesian(ylim = c(0, 4))
    }
    else if(type == 6) {
        p <- p + coord_cartesian(ylim = c(0, 1))
    }
        
    if(hide_legend) {
        p <- p + theme(legend.position="none")
    }
    
    return(p)
}

In [ ]:
results_prior['maxs']

In [ ]:
data['maxs']

### DEBUG: predictions without fitting

In [ ]:
l = GPflow.kernels.Linear(1)
p = GPflow.kernels.PeriodicKernel(1)
r = GPflow.kernels.RBF(1)

In [ ]:
l.predict(0)

In [ ]:
X = np.linspace(31, 365*4, int(365*4-31+1))[:,None]

model = GPflow.gpr.GPR(X, Y, kern = kernel)

l.predict_y(X)

TypeError: list indices must be integers or slices, not float